In [1]:
from utility_dfcounter import *
from utility_dfcutter import *

# 1. count nominal

In [2]:
def countDataFrames(variation=""):
    labels  = ["trigger","usetag","acc","accVar","nmcbg","nmcbgVar","nfake","nfakeVar","ndata","ndataVar"]
    records = []
    
    for trigger in ["mu","e"]:
        for usetag in ["1b","2b"]:
            
            print( "counting "+trigger+usetag + " ...")

            counter = DFCounter(trigger,usetag)
            counter.setVariation(variation)

            acc,accVar = counter.returnAcc()
            nmcbg,nmcbgVar = counter.returnNMCbg()
            nfake,nfakeVar = counter.returnNFake()
            ndata,ndataVar = counter.returnNData()
            records.append( (trigger,usetag,acc,accVar,nmcbg,nmcbgVar,nfake,nfakeVar,ndata,ndataVar) )

    df = pd.DataFrame.from_records(records, columns=labels)
    df.to_pickle( common.getBaseDirectory() + "data/counts/count_{}.pkl".format(variation))
    print( "counting finished!")

In [3]:
#countDataFrames()

# 2. count with variation

In [4]:
import multiprocessing as mp

def runCountDataFrames(vlist):
    processes = []
    for v in vlist:
        processes.append(mp.Process(target=countDataFrames,args=(v,)))
    for ps in processes:
        ps.start()
    for ps in processes:
        ps.join()

In [5]:
runCountDataFrames(["PDFUp","PDFDown","TTXSUp","TWXSUp"]) # ttLHEWeight
runCountDataFrames(['EPtDown','MuPtDown','TauPtDown',""]) # lepEnergy
runCountDataFrames(["JESUp","JESDown","JERUp","JERDown"]) # jetEnergy
runCountDataFrames(["BTagUp","BTagDown","MistagUp","MistagDown"]) # bTagging

# runCountDataFrames(['FSRUp','FSRDown','ISRUp','ISRDown']) # ttTheory
# runCountDataFrames(['UEUp','UEDown','MEPSUp','MEPSDown']) # ttTheory
# runCountDataFrames(["RenormUp","RenormDown","FactorUp","FactorDown"]) # ttLHEWeight

counting mu1b ...
counting mu1b ...
counting mu1b ...
counting mu1b ...
counting mu2b ...
counting mu2b ...
counting mu2b ...
counting mu2b ...
counting e1b ...
counting e1b ...
counting e1b ...
counting e1b ...
counting e2b ...
counting e2b ...
counting e2b ...
counting e2b ...
counting finished!
counting finished!
counting finished!
counting finished!
counting mu1b ...
counting mu1b ...
counting mu1b ...
counting mu1b ...
counting mu2b ...
counting mu2b ...
counting mu2b ...
counting mu2b ...
counting e1b ...
counting e1b ...
counting e1b ...
counting e1b ...
counting e2b ...
counting e2b ...
counting e2b ...
counting e2b ...
counting finished!
counting finished!
counting finished!
counting finished!
counting mu1b ...
counting mu1b ...
counting mu1b ...
counting mu1b ...
counting mu2b ...
counting mu2b ...
counting mu2b ...
counting mu2b ...
counting e1b ...
counting e1b ...
counting e1b ...
counting e1b ...
counting e2b ...
counting e2b ...
counting e2b ...
counting e2b ...
counting

### [Backup] Count uncertainty of Tau acc

In [4]:
# jet misID
df = DFCutter('etau','>1',"mctt").getDataFrame()
np.sum( df.eventWeight*(1+0.002*df.lepton2_pt) )/ np.sum(df.eventWeight)

1.0798866495826887

### [Backup] Print the Expect Yields Table

In [5]:
ct = DFCounter('e','1b')
selection = 'etau'

total,totalVar = 0,0


df = DFCutter(selection,ct.nbjet,"mctt").getDataFrame()
n,nVar = np.sum(df.eventWeight),np.sum(df.eventWeight**2)
print("{:8.1f} +/- {:4.1f}".format(n,nVar**0.5) )
total += n
totalVar += nVar

df = DFCutter(selection,ct.nbjet,"mct").getDataFrame()
n,nVar = np.sum(df.eventWeight),np.sum(df.eventWeight**2)
print("{:8.1f} +/- {:4.1f}".format(n,nVar**0.5) )
total += n
totalVar += nVar

df = DFCutter(selection,ct.nbjet,"mcw").getDataFrame()
n,nVar = np.sum(df.eventWeight),np.sum(df.eventWeight**2)
print("{:8.1f} +/- {:4.1f}".format(n,nVar**0.5) )
total += n
totalVar += nVar

df = DFCutter(selection,ct.nbjet,"mcz").getDataFrame()
n,nVar = np.sum(df.eventWeight),np.sum(df.eventWeight**2)
print("{:8.1f} +/- {:4.1f}".format(n,nVar**0.5) )
total += n
totalVar += nVar

df = DFCutter(selection,ct.nbjet,"mcdiboson").getDataFrame()
n,nVar = np.sum(df.eventWeight),np.sum(df.eventWeight**2)
print("{:8.1f} +/- {:4.1f}".format(n,nVar**0.5) )
total += n
totalVar += nVar

n, nVar = ct.getNFake(selection,ct.nbjet)
print("{:8.1f} +/- {:4.1f}".format(n,nVar**0.5) )
total += n
totalVar += nVar

print("{:8.1f} +/- {:4.1f}".format(total,totalVar**0.5) )
df = DFCutter(selection,ct.nbjet,"data2016").getDataFrame()
print("{:8.1f} +/- {:4.1f}".format(np.sum(df.eventWeight),np.sum(df.eventWeight**2)**0.5) )

 11663.0 +/- 45.3
   710.4 +/- 29.3
   178.6 +/- 20.6
   325.6 +/- 51.1
    15.5 +/-  1.2
   518.6 +/- 64.1
 13411.7 +/- 100.3
 13225.0 +/- 115.0
